In [1]:
import keras
import pandas as pd
from keras import layers
import tensorflow as tf

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
import numpy as np
import os

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error as mse
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.layers import Dense, InputLayer, LSTM, Dropout, Bidirectional, GRU

# from keras import ops
import numpy as np

In [3]:
DC_POWER_INDEX = 0
LEARNING_RATE = 0.001
RANDOM_STATE = 44
START_TIME = '2020-06-07 00:30:00'
END_TIME = '2020-06-07 04:00:00'
BATCH_SIZE = 1024

def fix_randomness():
    tf.random.set_seed(RANDOM_STATE)

    from numpy.random import seed
    seed(RANDOM_STATE)
    keras.utils.set_random_seed(RANDOM_STATE)

fix_randomness()

MULTIVARIATE

In [4]:
EPOCHS = 500
MULTIV_TRAINING_WINDOW = 40
lead_time = 20

In [5]:
# Creating the model input for multivariate forecasting
def df_to_model_input2(df_np, col_index, window_size):
    df_np = df_np.to_numpy()
    x = []
    y = []

    for i in range(len(df_np) - window_size - lead_time):
        row = [r for r in df_np[i:i+window_size]]
        x.append(row)

        label = df_np[i+lead_time+window_size][col_index]
        y.append(label)

    return np.array(x), np.array(y)

In [6]:
multiv_teacher_path = 'models/girasol/multiv_teacher_model.keras'
multiv_student_path = 'models/girasol/multiv_student_model.keras'
base_model_path = 'models/girasol/base_model.keras'

In [7]:
train = pd.read_csv('/.../girasol_met/train.csv')
test = pd.read_csv('/.../girasol_met/val.csv')

In [8]:
NUMBER_OF_FEATURES = len(train.columns)

In [9]:
scaler = MinMaxScaler()
train = pd.DataFrame(scaler.fit_transform(train), columns=train.columns)
test = pd.DataFrame(scaler.transform(test), columns=test.columns)

In [10]:
def get_train_test_val(x, y):
    return x[:train_len], y[:train_len], x[train_len:], y[train_len:]

x, y = df_to_model_input2(train, 7, MULTIV_TRAINING_WINDOW)

ds_len = len(y)
train_len = int(0.8*ds_len)

x_train, y_train, x_val, y_val = get_train_test_val(x, y)
x_test, y_test = df_to_model_input2(test, 7, MULTIV_TRAINING_WINDOW)

In [11]:
x_test.shape,  y_test.shape

((25163, 40, 8), (25163,))

In [12]:
teacher_hidden_layer_units, teacher_dense_params = 512, 8
student_hidden_layer_units, student_dense_params = 16, 8

In [13]:
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

def calculate_mape(scaler, y_pred_scaled, target_column_index=-1):

    # Create dummy arrays to match the scaler's expected input shape
    dummy_data_pred = np.zeros((y_pred_scaled.shape[0], NUMBER_OF_FEATURES))
    dummy_data_test = np.zeros((y_test.shape[0], NUMBER_OF_FEATURES))

    # Insert the predicted values and y_test into the correct column
    dummy_data_pred[:, target_column_index] = y_pred_scaled[:, 0]
    dummy_data_test[:, target_column_index] = y_test[:] # it kept that way to be able to modify if needed

    # Perform inverse transform to get the unscaled predictions and y_test
    unscaled_predictions = scaler.inverse_transform(dummy_data_pred)[:, target_column_index]
    y_test_unscaled = scaler.inverse_transform(dummy_data_test)[:, target_column_index]



    mape = mean_absolute_percentage_error(y_test_unscaled, unscaled_predictions)*100
    return mape

def calculate_metrics(y_true, y_pred):
    """
    Calculate NMSE, NRMSE, and NMAE normalized by the average of true values.
    
    Parameters:
        y_true (array-like): True values.
        y_pred (array-like): Predicted values.
        
    Returns:
        dict: A dictionary containing NMSE, NRMSE, and NMAE.
    """
    # Convert inputs to numpy arrays for consistency
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Mean Squared Error (MSE) and Mean Absolute Error (MAE) using sklearn
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    mape = calculate_mape(scaler=scaler, y_pred_scaled=y_pred)
    # Variance and mean of true values
    avg_true = np.mean(y_true)
    
    # Calculate metrics
    nrmse = np.sqrt(mse) / avg_true
    nmae = mae / avg_true

    print(f"NRMSE (Normalized Root Mean Squared Error): {nrmse:.4f}")
    print(f"NMAE (Normalized Mean Absolute Error): {nmae:.4f}")
    print(f"MAPE (Mean Absolute Percentage Error): {mape:.4f}") 
    
    # Return metrics as a dictionary
    return nrmse, nmae, mape

In [14]:
from sklearn.preprocessing import MinMaxScaler

RANDOM_STATE = 44
tf.random.set_seed(RANDOM_STATE)
from numpy.random import seed
seed(RANDOM_STATE)
keras.utils.set_random_seed(RANDOM_STATE)

def run_teacher_multiv():

    # Create the model
    model = Sequential()
    model.add(InputLayer((MULTIV_TRAINING_WINDOW, NUMBER_OF_FEATURES)))
    model.add(GRU(teacher_hidden_layer_units))
    model.add(Dropout(0.5))
    model.add(Dense(teacher_dense_params, 'relu'))
    model.add(Dense(1, 'linear'))

    # Model checkpoint
    model_file = multiv_teacher_path

    cp = ModelCheckpoint(model_file, save_best_only=True)

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=5,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True  # Restore the weights of the best epoch after stopping
    )

    # Compile the model
    model.compile(
        loss=MeanSquaredError(),
        optimizer=Adam(learning_rate=LEARNING_RATE),
        metrics=[MeanSquaredError()]
    )
   
    # Train the model
    model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=[cp, early_stopping]
    )

run_teacher_multiv()

teacher = load_model(multiv_teacher_path)
print("Teacher Validation RMSE") 
teacher.evaluate(x_val, y_val)

Epoch 1/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 695ms/step - loss: 0.0563 - mean_squared_error: 0.0563 - val_loss: 0.0066 - val_mean_squared_error: 0.0065
Epoch 2/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 710ms/step - loss: 0.0090 - mean_squared_error: 0.0090 - val_loss: 0.0052 - val_mean_squared_error: 0.0055
Epoch 3/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 718ms/step - loss: 0.0078 - mean_squared_error: 0.0078 - val_loss: 0.0048 - val_mean_squared_error: 0.0052
Epoch 4/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 27s 718ms/step - loss: 0.0074 - mean_squared_error: 0.0074 - val_loss: 0.0050 - val_mean_squared_error: 0.0053
Epoch 5/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 707ms/step - loss: 0.0070 - mean_squared_error: 0.0070 - val_loss: 0.0048 - val_mean_squared_error: 0.0052
Epoch 6/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 713ms/step - loss: 0.0069 - mean_squared_error: 0.0069 - val_loss: 0.0046 - val_mean_squared_error: 0.0050
Epoch 7/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 715ms/step - loss: 0.0067 - mean_squared_error: 0.0067 - 

[0.004465042147785425, 0.004489721264690161]

In [15]:
teacher = load_model(multiv_teacher_path)

# Make predictions on the scaled test set
y_pred = teacher.predict(x_test)

nrmse, nmae, mape = calculate_metrics(y_test, y_pred)
# Store the results in a dictionary for the teacher
teacher_results = {
    "NRMSE": nrmse,
    "NMAE": nmae,
    "MAPE": mape
}

# Print the results in a single row
print(f"Teacher Results: NRMSE = {teacher_results['NRMSE']:.4f}, NMAE = {teacher_results['NMAE']:.4f}, MAPE = {teacher_results['MAPE']:.4f}")


787/787 ━━━━━━━━━━━━━━━━━━━━ 46s 58ms/step
NRMSE (Normalized Root Mean Squared Error): 0.1638
NMAE (Normalized Mean Absolute Error): 0.1099
MAPE (Mean Absolute Percentage Error): 16.0381
Teacher Results: NRMSE = 0.1638, NMAE = 0.1099, MAPE = 16.0381


In [16]:
from sklearn.preprocessing import MinMaxScaler

RANDOM_STATE = 44
tf.random.set_seed(RANDOM_STATE)
from numpy.random import seed
seed(RANDOM_STATE)
keras.utils.set_random_seed(RANDOM_STATE)

def run_student_multiv():

    # Create the model
    model = Sequential()
    model.add(InputLayer((MULTIV_TRAINING_WINDOW, NUMBER_OF_FEATURES)))
    model.add(GRU(student_hidden_layer_units))
    model.add(Dropout(0.5))
    model.add(Dense(student_dense_params, 'relu'))
    model.add(Dense(1, 'linear'))

    # Model checkpoint
    model_file = multiv_student_path

    cp = ModelCheckpoint(model_file, save_best_only=True)

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=5,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True  # Restore the weights of the best epoch after stopping
    )

    # Compile the model
    model.compile(
        loss=MeanSquaredError(),
        optimizer=Adam(learning_rate=LEARNING_RATE),
        metrics=[MeanSquaredError()]
    )
   
    # Train the model
    model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        batch_size=BATCH_SIZE,
        epochs=500,
        callbacks=[cp, early_stopping]
    )

run_student_multiv()

student = load_model(multiv_student_path)
print("Student Validation RMSE") 
student.evaluate(x_val, y_val)

Epoch 1/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.1493 - mean_squared_error: 0.1492 - val_loss: 0.0366 - val_mean_squared_error: 0.0374
Epoch 2/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0571 - mean_squared_error: 0.0571 - val_loss: 0.0267 - val_mean_squared_error: 0.0287
Epoch 3/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0406 - mean_squared_error: 0.0406 - val_loss: 0.0188 - val_mean_squared_error: 0.0202
Epoch 4/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0326 - mean_squared_error: 0.0326 - val_loss: 0.0142 - val_mean_squared_error: 0.0147
Epoch 5/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0264 - mean_squared_error: 0.0264 - val_loss: 0.0104 - val_mean_squared_error: 0.0104
Epoch 6/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0218 - mean_squared_error: 0.0218 - val_loss: 0.0086 - val_mean_squared_error: 0.0084
Epoch 7/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0185 - mean_squared_error: 0.0185 - val_loss: 0.00

[0.005269063636660576, 0.00529178511351347]

In [17]:
student = load_model(multiv_student_path)

# Make predictions on the scaled test set
y_pred = student.predict(x_test)

nrmse, nmae, mape = calculate_metrics(y_test, y_pred)

# Store the results in a dictionary
student_results = {
    "NRMSE": nrmse,
    "NMAE": nmae,
    "MAPE": mape
}
# Print the results in a single row
print(f"Student Results: NRMSE = {student_results['NRMSE']:.4f}, NMAE = {student_results['NMAE']:.4f}, MAPE = {student_results['MAPE']:.4f}")


787/787 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
NRMSE (Normalized Root Mean Squared Error): 0.1977
NMAE (Normalized Mean Absolute Error): 0.1585
MAPE (Mean Absolute Percentage Error): 23.9094
Student Results: NRMSE = 0.1977, NMAE = 0.1585, MAPE = 23.9094


In [18]:
from tensorflow.keras.saving import register_keras_serializable

@register_keras_serializable()
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student
        self._loss_tracker = keras.metrics.Mean(name="loss")

    def compile(
        self,
        optimizer,
        loss,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        loss_threshold,
        alpha,
        temperature

    ):
        """Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super().compile(loss=loss, optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature
        self.loss_threshold = loss_threshold

    def get_config(self):
        config = {
            'student': self.student.to_json(),
            'teacher': self.teacher.to_json()
        }

        return config

    @classmethod
    def from_config(cls, config):
        student = tf.keras.models.model_from_json(config.pop('student'))
        teacher = tf.keras.models.model_from_json(config.pop('teacher'))
        return cls(student=student, teacher=teacher, **config)

    def train_step(self, data):
        # Unpack the data
        x, y = data

        with tf.GradientTape() as tape:
            # Forward pass
            y_pred = self.student(x, training=True)
            # Compute the loss value
            loss = self.compute_loss(x, y, y_pred)

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`
        self.compiled_metrics.update_state(y, y_pred)

        # Return a dict mapping metric names to current value
        results =  {m.name: m.result() for m in self.metrics}

        results['total loss (train)'] = loss if isinstance(loss, float) else tf.reduce_mean(loss)

        return results


    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Forward pass to get student's predictions
        y_pred = self.student(x, training=False)

        # Compute the validation loss
        val_loss = self.compute_loss(x, y, y_pred)

        # Manually update the metrics for validation
        self.compiled_metrics.update_state(y, y_pred)

        # Collect results for all metrics
        results = {m.name: m.result() for m in self.metrics}

        # Ensure 'val_loss' is properly reduced to a scalar and reported
        results['total loss (val)'] = val_loss if isinstance(val_loss, float) else tf.reduce_mean(val_loss)

        return results



    def compute_loss(
        # self, x=None, y=None, y_pred=None, sample_weight=None, allow_empty=False
        self, x=None, y=None, y_pred=None, sample_weight=None, allow_empty=False

    ):

        mse = MeanSquaredError()

        # Compute predictions by the teacher model
        teacher_pred = self.teacher(x, training=False)

        # Compute the MSE loss between true labels and student predictions
        student_loss = mse(y, y_pred)

        # Teacher loss is the
        temp = self.temperature
        # loss = self.alpha * student_loss + (1 - self.alpha) * teacher_loss
        loss = self.alpha * student_loss + (1 - self.alpha) * (mse(y_pred/temp, teacher_pred/temp)* (temp ** 2))


        return loss

    def call(self, x):
        return self.student(x)

In [19]:
def run_multiv_distillation(teacher, alpha, threshold, temperature, file_name, window_size):

    # Create the model
    model = Sequential()
    model.add(InputLayer((MULTIV_TRAINING_WINDOW, NUMBER_OF_FEATURES)))
    model.add(GRU(student_hidden_layer_units))
    model.add(Dropout(0.5))
    model.add(Dense(student_dense_params, 'relu'))
    model.add(Dense(1, 'linear'))

    # Create the distiller class
    distiller = Distiller(student=model, teacher=teacher)

    # Compile the distiller class
    distiller.compile(
    optimizer = Adam(learning_rate=LEARNING_RATE),
    loss = MeanSquaredError(),
    metrics = [RootMeanSquaredError()],
    student_loss_fn = MeanSquaredError(),
    distillation_loss_fn = distiller.compute_loss, #MeanSquaredError(),
    loss_threshold = threshold,
    alpha=alpha,
    temperature=temperature
    )

    dummy_x = tf.random.normal([1, *((window_size, NUMBER_OF_FEATURES))])  # Replace `input_shape` with the actual shape of your input
    _ = distiller(dummy_x) 

    d_check = ModelCheckpoint(file_name, monitor='root_mean_squared_error',save_best_only=True)

    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=5,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True  # Restore the weights of the best epoch after stopping
    )

    distiller.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=EPOCHS, callbacks=[d_check, early_stopping])

    distilled_student = load_model(file_name)
    # Make predictions using the distilled student model
    y_pred = distilled_student.predict(x_val)

    # Calculate the root mean squared error (RMSE)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print("Distilled Student RMSE:", rmse)

In [20]:
alphas = [0.3, 0.5, 0.7, 0.9]
temps = [5]
threshold = 0

# Initialize a 2D list to store RMSE values
rmse_matrix = []

# Iterate over alphas and temps
for alpha in alphas:
    row_rmse = []
    for temp in temps:

        RANDOM_STATE = 44
        tf.random.set_seed(RANDOM_STATE)
        from numpy.random import seed
        seed(RANDOM_STATE)
        keras.utils.set_random_seed(RANDOM_STATE)

        multiv_distill_file_name = 'models/girasol/distils/multiv_distil_'+str(alpha)+'_'+str(temp)+'/multiv_distil.keras'
        
        run_multiv_distillation(teacher, alpha, threshold, temp,  multiv_distill_file_name, MULTIV_TRAINING_WINDOW)
        
        # Load the model
        multiv_distill_file_name = f'models/girasol/distils/multiv_distil_{alpha}_{temp}/multiv_distil.keras'
        student_kd = load_model(multiv_distill_file_name)

        # Make predictions on the scaled test set
        y_pred = student_kd.predict(x_test)

        nrmse, nmae, mape = calculate_metrics(y_test, y_pred)
        
        row_rmse.append(nrmse)

    # Append the row of RMSE values to the matrix
    rmse_matrix.append(row_rmse)

# Convert the list to a NumPy array for plotting
rmse_matrix = np.array(rmse_matrix)

Epoch 1/500


/Users/ifranrahmannijhum/miniforge3/envs/tf/lib/python3.12/site-packages/keras/src/backend/tensorflow/trainer.py:639: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


1162/1162 ━━━━━━━━━━━━━━━━━━━━ 87s 74ms/step - root_mean_squared_error: 0.1959 - loss: 0.5017 - total loss (train): 0.0174 - val_loss: 0.5516 - val_total loss (val): 0.0057
Epoch 2/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 81s 70ms/step - root_mean_squared_error: 0.0932 - loss: 0.5298 - total loss (train): 0.0043 - val_loss: 0.5185 - val_total loss (val): 0.0038
Epoch 3/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 87s 75ms/step - root_mean_squared_error: 0.0854 - loss: 0.5301 - total loss (train): 0.0032 - val_loss: 0.4758 - val_total loss (val): 0.0038
Epoch 4/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 82s 71ms/step - root_mean_squared_error: 0.0822 - loss: 0.5300 - total loss (train): 0.0028 - val_loss: 0.4407 - val_total loss (val): 0.0046
Epoch 5/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 81s 70ms/step - root_mean_squared_error: 0.0806 - loss: 0.5298 - total loss (train): 0.0026 - val_loss: 0.4162 - val_total loss (val): 0.0045
Epoch 6/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 77s 66ms/step - root_mean_squared_error: 0.0

/Users/ifranrahmannijhum/miniforge3/envs/tf/lib/python3.12/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


291/291 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Distilled Student RMSE: 0.07225228867958118
120/787 ━━━━━━━━━━━━━━━━━━━━ 0s 844us/step

...

1162/1162 ━━━━━━━━━━━━━━━━━━━━ 46s 39ms/step - root_mean_squared_error: 0.1960 - loss: 0.4984 - total loss (train): 0.0210 - val_loss: 0.5482 - val_total loss (val): 0.0126
Epoch 2/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 44s 38ms/step - root_mean_squared_error: 0.0981 - loss: 0.5260 - total loss (train): 0.0086 - val_loss: 0.5676 - val_total loss (val): 0.0152
Epoch 3/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 43s 37ms/step - root_mean_squared_error: 0.0905 - loss: 0.5262 - total loss (train): 0.0073 - val_loss: 0.5494 - val_total loss (val): 0.0185
Epoch 4/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 43s 37ms/step - root_mean_squared_error: 0.0850 - loss: 0.5261 - total loss (train): 0.0066 - val_loss: 0.5535 - val_total loss (val): 0.0209
Epoch 5/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 42s 36ms/step - root_mean_squared_error: 0.0833 - loss: 0.5261 - total loss (train): 0.0063 - val_loss: 0.5464 - val_total loss (val): 0.0187
Epoch 6/500
1162/1162 ━━━━━━━━━━━━━━━━━━━━ 43s 37ms/step - root_mean_squared_error: 0.0

/Users/ifranrahmannijhum/miniforge3/envs/tf/lib/python3.12/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


291/291 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Distilled Student RMSE: 0.06837722226837102
126/787 ━━━━━━━━━━━━━━━━━━━━ 0s 808us/step

/Users/ifranrahmannijhum/miniforge3/envs/tf/lib/python3.12/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


787/787 ━━━━━━━━━━━━━━━━━━━━ 1s 899us/step
NRMSE (Normalized Root Mean Squared Error): 0.1705
NMAE (Normalized Mean Absolute Error): 0.1171
MAPE (Mean Absolute Percentage Error): 19.7833


In [21]:
import numpy as np
from tensorflow.keras.models import load_model

# Example inputs
alphas = [0.3, 0.5, 0.7, 0.9]
temps = [5]
threshold = 0

# Initialize a dictionary to store results
student_kd_results = []

# Iterate over alphas and temps
for alpha in alphas:
    for temp in temps:
        multiv_distill_file_name = f'models/girasol/distils/multiv_distil_{alpha}_{temp}/multiv_distil.keras'
        
        # Load the model
        student_kd = load_model(multiv_distill_file_name)

        # Make predictions on the scaled test set
        y_pred = student_kd.predict(x_test)

        # Calculate metrics
        nrmse, nmae, mape = calculate_metrics(y_test, y_pred)
        
        # Add the results to the dictionary
        student_kd_results.append({
            "alpha": alpha,
            "temp": temp,
            "NRMSE": nrmse,
            "NMAE": nmae,
            "MAPE": mape
        })

# Print the results
for result in student_kd_results:
    print(f"Alpha: {result['alpha']}, Temp: {result['temp']}, NRMSE: {result['NRMSE']:.4f}, "
          f"NMAE: {result['NMAE']:.4f}, MAPE: {result['MAPE']:.4f}")

125/787 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step

/Users/ifranrahmannijhum/miniforge3/envs/tf/lib/python3.12/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


787/787 ━━━━━━━━━━━━━━━━━━━━ 1s 899us/step
NRMSE (Normalized Root Mean Squared Error): 0.1804
NMAE (Normalized Mean Absolute Error): 0.1353
MAPE (Mean Absolute Percentage Error): 20.8718
127/787 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step

/Users/ifranrahmannijhum/miniforge3/envs/tf/lib/python3.12/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


787/787 ━━━━━━━━━━━━━━━━━━━━ 1s 894us/step
NRMSE (Normalized Root Mean Squared Error): 0.1767
NMAE (Normalized Mean Absolute Error): 0.1292
MAPE (Mean Absolute Percentage Error): 20.8544
124/787 ━━━━━━━━━━━━━━━━━━━━ 0s 820us/step

/Users/ifranrahmannijhum/miniforge3/envs/tf/lib/python3.12/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


787/787 ━━━━━━━━━━━━━━━━━━━━ 1s 898us/step
NRMSE (Normalized Root Mean Squared Error): 0.1688
NMAE (Normalized Mean Absolute Error): 0.1151
MAPE (Mean Absolute Percentage Error): 19.0298
125/787 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step

/Users/ifranrahmannijhum/miniforge3/envs/tf/lib/python3.12/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


787/787 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
NRMSE (Normalized Root Mean Squared Error): 0.1705
NMAE (Normalized Mean Absolute Error): 0.1171
MAPE (Mean Absolute Percentage Error): 19.7833
Alpha: 0.3, Temp: 5, NRMSE: 0.1804, NMAE: 0.1353, MAPE: 20.8718
Alpha: 0.5, Temp: 5, NRMSE: 0.1767, NMAE: 0.1292, MAPE: 20.8544
Alpha: 0.7, Temp: 5, NRMSE: 0.1688, NMAE: 0.1151, MAPE: 19.0298
Alpha: 0.9, Temp: 5, NRMSE: 0.1705, NMAE: 0.1171, MAPE: 19.7833


RESULTS

Efficiency Metrics

In [22]:
teacher = load_model(multiv_teacher_path)

# 1. Print model summary (Layer details and parameter count)
print("Model Summary:")
teacher.summary()

# 2. Get the total number of parameters
total_params = teacher.count_params()
print(f'\nTotal number of parameters: {total_params}')

# 3. Get the size of the model file on disk
model_size = os.path.getsize(multiv_teacher_path) / 1024  #a Convert from bytes to MB
print(f'Model size on disk: {model_size:.2f} KB')

Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 512)            │       801,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │         4,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,417,717 (9.22 MB)

 Trainable params: 805,905 (3.07 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,611,812 (6.15 MB)


Total number of parameters: 805905
Model size on disk: 9474.91 KB


In [23]:
student = load_model(multiv_student_path)

# 1. Print model summary (Layer details and parameter count)
print("Model Summary:")
student.summary()

# 2. Get the total number of parameters
total_params = student.count_params()
print(f'\nTotal number of parameters: {total_params}')

# 3. Get the size of the model file on disk
model_size = os.path.getsize(multiv_student_path) / 1024  #a Convert from bytes to MB
print(f'Model size on disk: {model_size:.2f} KB')

Model Summary:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_1 (GRU)                     │ (None, 16)             │         1,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,181 (16.34 KB)

 Trainable params: 1,393 (5.44 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,788 (10.89 KB)


Total number of parameters: 1393
Model size on disk: 47.04 KB


Error Metrics

In [24]:
# Print the results in a single row
print(f"Teacher Results: NRMSE = {teacher_results['NRMSE']:.4f}, NMAE = {teacher_results['NMAE']:.4f}, MAPE = {teacher_results['MAPE']:.4f}")

# Print the results in a single row
print(f"Student Results: NRMSE = {student_results['NRMSE']:.4f}, NMAE = {student_results['NMAE']:.4f}, MAPE = {student_results['MAPE']:.4f}")

print('Student KD Results')
# Print the results
for result in student_kd_results:
    print(f"Alpha: {result['alpha']}, Temp: {result['temp']}, NRMSE: {result['NRMSE']:.4f}, "
          f"NMAE: {result['NMAE']:.4f}, MAPE: {result['MAPE']:.4f}")

Teacher Results: NRMSE = 0.1638, NMAE = 0.1099, MAPE = 16.0381
Student Results: NRMSE = 0.1977, NMAE = 0.1585, MAPE = 23.9094
Student KD Results
Alpha: 0.3, Temp: 5, NRMSE: 0.1804, NMAE: 0.1353, MAPE: 20.8718
Alpha: 0.5, Temp: 5, NRMSE: 0.1767, NMAE: 0.1292, MAPE: 20.8544
Alpha: 0.7, Temp: 5, NRMSE: 0.1688, NMAE: 0.1151, MAPE: 19.0298
Alpha: 0.9, Temp: 5, NRMSE: 0.1705, NMAE: 0.1171, MAPE: 19.7833


Forecasting Skill

In [25]:
alpha = 0.9
temp = 5
distilled_student_path = f'models/girasol/distils/multiv_distil_{alpha}_{temp}/multiv_distil.keras'

# Load models
multiv_teacher = load_model(multiv_teacher_path)
multiv_student = load_model(multiv_student_path)
base_model = load_model(base_model_path)
distilled_student = load_model(distilled_student_path)

# Function to calculate RMSE
def calculate_rmse(model, x_val, y_val):
    y_pred = model.predict(x_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    return rmse

# Calculate RMSE for all models
rmse_multiv_teacher = calculate_rmse(multiv_teacher, x_val, y_val)
rmse_multiv_student = calculate_rmse(multiv_student, x_val, y_val)
rmse_base_model = calculate_rmse(base_model, x_val, y_val)
rmse_distilled_student = calculate_rmse(distilled_student, x_val, y_val)

# Calculate forecasting skill scores relative to the base model
def calculate_skill_score(base_rmse, model_rmse):
    return 100 * (1 - model_rmse / base_rmse)

skill_teacher = calculate_skill_score(rmse_base_model, rmse_multiv_teacher)
skill_student = calculate_skill_score(rmse_base_model, rmse_multiv_student)
skill_distilled = calculate_skill_score(rmse_base_model, rmse_distilled_student)

# Print RMSE and skill scores in a clear format
print(f"{'Model':<20}{'RMSE':<10}{'Forecasting Skill (%)':<25}")
print(f"{'-'*55}")
print(f"{'Multiv Teacher':<20}{rmse_multiv_teacher:<10.4f}{skill_teacher:<25.2f}")
print(f"{'Multiv Student':<20}{rmse_multiv_student:<10.4f}{skill_student:<25.2f}")
print(f"{'Base Model':<20}{rmse_base_model:<10.4f}{'--':<25}")
print(f"{'Distilled Student':<20}{rmse_distilled_student:<10.4f}{skill_distilled:<25.2f}")

  4/291 ━━━━━━━━━━━━━━━━━━━━ 7s 25ms/step 

/Users/ifranrahmannijhum/miniforge3/envs/tf/lib/python3.12/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


291/291 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step
291/291 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
  1/291 ━━━━━━━━━━━━━━━━━━━━ 1:00 209ms/step

I0000 00:00:1736791091.874953  144074 service.cc:148] XLA service 0x309c261f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1736791091.875098  144074 service.cc:156]   StreamExecutor device (0): Host, Default Version
2025-01-13 17:58:11.898303: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1736791092.051277  144074 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


291/291 ━━━━━━━━━━━━━━━━━━━━ 0s 303us/step
291/291 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Model               RMSE      Forecasting Skill (%)    
-------------------------------------------------------
Multiv Teacher      0.0668    1.88                     
Multiv Student      0.0726    -6.59                    
Base Model          0.0681    --                       
Distilled Student   0.0684    -0.41                    
